# Install Modules

In [1]:
pip install torch

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


# Import necessary libraries

In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# Define the differential equation for the liquid state

In [8]:
class LiquidTimeStep(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(LiquidTimeStep, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.W_in = nn.Linear(input_size, hidden_size)
        self.W_h = nn.Linear(hidden_size, hidden_size)
        self.tau = nn.Parameter(torch.ones(hidden_size))
    
    def forward(self, x, h):
        dx = torch.tanh(self.W_in(x) + self.W_h(h))
        h_new = h + (dx - h) / self.tau
        return h_new

# Define the Liquid Neural Network

In [9]:
class LiquidNeuralNetwork(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(LiquidNeuralNetwork, self).__init__()
        self.hidden_size = hidden_size
        self.liquid_step = LiquidTimeStep(input_size, hidden_size)
        self.output_layer = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        batch_size, seq_len, _ = x.size()
        h = torch.zeros(batch_size, self.hidden_size, device=x.device)
        for t in range(seq_len):
            h = self.liquid_step(x[:, t, :], h)
        output = self.output_layer(h)
        return output

# Define parameters

In [10]:
# Hyperparameters
input_size = 10  
hidden_size = 20  
output_size = 1  # Output size for regression

# Create the model
model = LiquidNeuralNetwork(input_size, hidden_size, output_size)

# Define Loss and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training Setup for Image Classification

In [11]:
# === Hyperparameters ===
input_size = 3          # RGB channels
hidden_size = 64
output_size = 10        # 10 classes
batch_size = 32
num_epochs = 5
learning_rate = 0.001

# === Data Loading ===
transform = transforms.Compose([
    transforms.ToTensor(),
])

train_data = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_data  = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
test_loader  = DataLoader(test_data, batch_size=batch_size, shuffle=False)

# === Initialize Model, Loss, Optimizer ===
model = LiquidNeuralNetwork(input_size, hidden_size, output_size)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)


100%|███████████████████████████████████████████████████████████████| 170498071/170498071 [00:32<00:00, 5219857.10it/s]


Extracting ./data\cifar-10-python.tar.gz to ./data
Files already downloaded and verified


# Training Loop

In [12]:
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for images, labels in train_loader:
        # images: [batch, 3, 32, 32]
        images = images.permute(0, 2, 3, 1)       # -> [batch, 32, 32, 3]
        images = images.reshape(images.size(0), -1, 3)  # -> [batch, 1024, 3]

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
    
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss/len(train_loader):.4f}")


Epoch [1/5], Loss: 2.1987
Epoch [2/5], Loss: 2.2100
Epoch [3/5], Loss: 2.2104
Epoch [4/5], Loss: 2.2904
Epoch [5/5], Loss: 2.2259


# Testing the Model

In [ ]:
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for images, labels in test_loader:
        images = images.permute(0, 2, 3, 1).reshape(images.size(0), -1, 3)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Test Accuracy: {100 * correct / total:.2f}%")


# Feeding New Images Later (Continual Learning)

In [ ]:
def fine_tune_model(model, new_data_loader, learning_rate=0.0005, epochs=3):
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    criterion = nn.CrossEntropyLoss()
    
    model.train()
    for epoch in range(epochs):
        total_loss = 0
        for images, labels in new_data_loader:
            images = images.permute(0, 2, 3, 1).reshape(images.size(0), -1, 3)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        print(f"Fine-tune Epoch [{epoch+1}/{epochs}], Loss: {total_loss/len(new_data_loader):.4f}")


# Run it

In [ ]:
# Suppose you have new images (new_train_loader)
fine_tune_model(model, new_train_loader)

# Saving and Loading the Model

In [ ]:
# Save
torch.save(model.state_dict(), "liquid_model.pth")

# Load later
model = LiquidNeuralNetwork(input_size, hidden_size, output_size)
model.load_state_dict(torch.load("liquid_model.pth"))
model.eval()
